<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-11 14:13:36
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.36 C
-------------------
Today PnL: -93.01 K (-0.68%)
Current PnL: -13.86 L (-9.65%)
CY Booked + Current PnL: -6.90 L (-4.8%)
-------------------
Total profit:  4.75 L
Total loss:  -18.61 L
-------------------
Total Booked + Current PnL: 20.08 L (17.23%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.11%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.39 L (53.13%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-224.56,H-SC,96.88,89866.0,-5450.0,5491.0,2.69,-5.72,6.11,0.04,157.0,-0.99,0.66,77.25,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-21.55,H-LC,77.60,107920.0,3608.0,13932.0,-0.08,3.46,12.91,16.81,27.0,0.26,0.79,20.06,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,55.21,214120.0,12640.0,14582.0,-0.36,6.27,6.81,13.51,77.0,0.87,1.57,19.66,XY25,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,33.33,128304.0,-14175.0,14793.0,-0.36,-9.95,11.53,0.44,247.0,-0.96,0.94,51.43,OX40N,NTT,BANKS
4,APOLLOHOSP,6904.43,8285.00,-20.73,H-LC,82.29,108030.0,4464.0,16248.0,-2.13,4.31,15.04,20.00,32.0,0.27,0.79,18.99,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,NATIONALUM,189.63,247.44,-50.15,H-MC,59.38,100700.0,196.0,30442.0,0.56,0.2,30.23,30.49,97.0,0.01,0.74,32.66,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,HINDUNILVR,2413.81,2723.0,-9.74,X-LC,89.58,282822.0,12475.0,22145.0,4.85,4.61,7.83,12.81,5.0,0.56,2.08,18.28,XY25,NTT,FMCG
52,JPPOWER,18.73,26.2,-10.80,L-SC,100.00,179480.0,37638.0,18935.0,2.91,26.53,10.55,39.88,262.0,1.99,1.32,81.89,XY24,NTT,POWER
94,WHIRLPOOL,1167.49,2270.0,-39.06,M-SC,69.79,200831.0,29210.0,132850.0,2.85,17.02,66.15,94.43,201.0,0.22,1.47,48.23,XR,NTT,DURABLES
92,VIPIND,488.80,489.0,-224.56,H-SC,96.88,89866.0,-5450.0,5491.0,2.69,-5.72,6.11,0.04,157.0,-0.99,0.66,77.25,OX40N,NTT,MISC
56,LAOPALA,369.40,464.0,124.74,H-SC,32.29,71417.0,-29429.0,55255.0,2.61,-29.18,77.37,25.61,136.0,-0.53,0.52,28.80,AR,NTT,CERAMICS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,37.50,18087.0,-31506.0,825.0,-4.37,-63.53,4.56,-61.86,270.0,-38.19,0.13,21.30,XY24,ATH,MISC
83,TCS,3794.03,4998.00,-21.59,X-LC,13.54,255052.0,-40882.0,134795.0,-3.31,-13.81,52.85,31.73,1.0,-0.30,1.87,2.09,X40,BTT,IT
3,ALKYLAMINE,2347.98,4546.37,3.23,X-SC,95.83,96084.0,-4879.0,99409.0,-3.26,-4.83,103.46,93.63,67.0,-0.05,0.71,43.11,SR,ATH,CHEMICALS
69,SAMMAANCAP,170.35,326.00,-167.05,M-SC,44.79,76650.0,-25560.0,118953.0,-3.22,-25.01,155.19,91.37,212.0,-0.21,0.56,24.43,XY25,NTT,FINANCE
65,RELAXO,902.64,1176.00,-28.83,H-SC,97.92,64365.0,-51173.0,86165.0,-3.08,-44.29,133.87,30.28,131.0,-0.59,0.47,25.74,X40N,NTT,FOOTWEAR


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-224.56,H-SC,96.88,89866.0,-5450.0,5491.0,2.69,-5.72,6.11,0.04,157.0,-0.99,0.66,77.25,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,33.33,128304.0,-14175.0,14793.0,-0.36,-9.95,11.53,0.44,247.0,-0.96,0.94,51.43,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.99,M-SC,34.38,151226.0,-19915.0,19856.0,-0.49,-11.64,13.13,-0.03,191.0,-1.00,1.11,8.69,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,87.50,148998.0,-25561.0,25598.0,0.12,-14.64,17.18,0.02,133.0,-1.00,1.09,29.20,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-19.50,X-MC,5.73,84518.0,-18134.0,18129.0,-1.64,-17.67,21.45,-0.00,59.0,-1.00,0.62,28.48,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-21.59,X-LC,13.54,255052.0,-40882.0,134795.0,-3.31,-13.81,52.85,31.73,1.0,-0.30,1.87,2.09,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.91,X-LC,41.67,289228.0,27627.0,118005.0,-1.10,10.56,40.80,55.67,2.0,0.23,2.12,17.31,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-9.74,X-LC,89.58,282822.0,12475.0,22145.0,4.85,4.61,7.83,12.81,5.0,0.56,2.08,18.28,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,15.04,X-LC,71.88,263764.0,34531.0,32759.0,-2.45,15.06,12.42,29.35,10.0,1.05,1.94,25.33,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-7.40,X-LC,10.94,78192.0,-31980.0,121057.0,-2.00,-29.03,154.82,80.85,11.0,-0.26,0.57,15.16,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3166.8,NaN,NaN,98.96,79170.0,44721.0,0.0,0.84,129.82,0.00,129.82,274.0,inf,0.58,21.06,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-10.80,L-SC,100.00,179480.0,37638.0,18935.0,2.91,26.53,10.55,39.88,262.0,1.99,1.32,81.89,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,18.55,X-SC,92.71,252734.0,47257.0,34069.0,0.59,23.00,13.48,39.58,65.0,1.39,1.86,35.43,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,68.75,231747.0,6785.0,160879.0,-0.15,3.02,69.42,74.53,189.0,0.04,1.70,34.55,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,28.30,H-SC,50.00,131405.0,6108.0,53101.0,-2.38,4.87,40.41,47.25,143.0,0.12,0.96,64.40,MH,ATH,POWER
20,CAMS,3643.00,5250.99,1.87,H-SC,30.21,114444.0,12440.0,32582.0,-2.40,12.20,28.47,44.14,126.0,0.38,0.84,30.42,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-4.20,H-SC,93.75,179670.0,15000.0,54081.0,-0.71,9.11,30.10,41.95,124.0,0.28,1.32,17.26,X40N,ATH,AC
88,UNIONBANK,123.87,163.00,-6.01,M-LC,15.62,163842.0,23002.0,21496.0,0.17,16.33,13.12,31.59,88.0,1.07,1.20,42.32,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3166.80,NaN,NaN,98.96,79170.0,44721.0,0.0,0.84,129.82,0.00,129.82,274.0,inf,0.58,21.06,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-39.06,M-SC,69.79,200831.0,29210.0,132850.0,2.85,17.02,66.15,94.43,201.0,0.22,1.47,48.23,XR,NTT,DURABLES
54,KPIGREEN,497.21,732.16,28.30,H-SC,50.00,131405.0,6108.0,53101.0,-2.38,4.87,40.41,47.25,143.0,0.12,0.96,64.40,MH,ATH,POWER
45,INDIAMART,2327.09,4911.36,-47.14,H-SC,83.33,137720.0,14384.0,122585.0,0.48,11.66,89.01,111.05,123.0,0.12,1.01,34.94,AR,ATH,MISC
95,WIPRO,243.46,420.00,-6.04,M-LC,61.46,160797.0,9852.0,99598.0,-2.15,6.53,61.94,72.51,71.0,0.10,1.18,10.76,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,37.50,18087.0,-31506.0,825.0,-4.37,-63.53,4.56,-61.86,270.0,-38.19,0.13,21.30,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,10.94,66912.0,-46637.0,86691.0,-0.09,-41.07,129.56,35.27,265.0,-0.54,0.49,63.77,XR,NTT,HOTELS
32,ENRIN,1377.95,3166.80,NaN,NaN,98.96,79170.0,44721.0,0.0,0.84,129.82,0.00,129.82,274.0,inf,0.58,21.06,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.00,7183.33,M-SC,80.21,79927.0,-13883.0,90501.0,-0.28,-14.80,113.23,81.67,245.0,-0.15,0.59,54.78,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-224.56,H-SC,96.88,89866.0,-5450.0,5491.0,2.69,-5.72,6.11,0.04,157.0,-0.99,0.66,77.25,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-10.80,L-SC,100.00,179480.0,37638.0,18935.0,2.91,26.53,10.55,39.88,262.0,1.99,1.32,81.89,XY24,NTT,POWER
32,ENRIN,1377.95,3166.80,NaN,NaN,98.96,79170.0,44721.0,0.0,0.84,129.82,0.00,129.82,274.0,inf,0.58,21.06,AR,ATH,ELECTRICAL
65,RELAXO,902.64,1176.00,-28.83,H-SC,97.92,64365.0,-51173.0,86165.0,-3.08,-44.29,133.87,30.28,131.0,-0.59,0.47,25.74,X40N,NTT,FOOTWEAR
92,VIPIND,488.80,489.00,-224.56,H-SC,96.88,89866.0,-5450.0,5491.0,2.69,-5.72,6.11,0.04,157.0,-0.99,0.66,77.25,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,3.23,X-SC,95.83,96084.0,-4879.0,99409.0,-3.26,-4.83,103.46,93.63,67.0,-0.05,0.71,43.11,SR,ATH,CHEMICALS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.01
1,25,41.98
2,50,70.63


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.69
LC    33.60
MC    21.61
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.15
X40      12.82
XY25     12.13
X40N     11.85
XR        9.95
OX40N     8.21
AR        8.16
MH        1.70
X5K       1.42
X200      1.38
SR        1.19
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.48
M-SC    15.93
H-LC    13.93
X-LC    12.85
H-MC     9.60
X-MC     7.58
M-LC     5.78
X-SC     4.57
M-MC     4.07
L-SC     2.71
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.31,-0.40,34.03
IT,11.91,-11.93,66.89
FINANCE,8.48,-12.76,53.68
BANKS,7.66,-9.22,53.78
MISC,5.87,-20.08,76.41
PAINTS,5.64,-13.73,36.15
ELECTRICAL,5.44,0.60,46.47
HEALTHCARE,4.75,0.44,28.64
AUTO,4.28,-16.03,67.05


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2648142.0,26
XR,1100386.0,13
AR,1039283.0,10
X40,710874.0,10
X40N,547976.0,11
XY25,467384.0,8
OX40N,311953.0,11
SR,182608.0,2
MH,83543.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1993675.0,20
M-SC,1772625.0,21
H-LC,641687.0,12
X-LC,559509.0,8
H-MC,527961.0,8
X-MC,421786.0,6
M-MC,415888.0,3
X-SC,297855.0,5
M-LC,273229.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      902839.0      6
M-SC       XY24      806374.0      7
H-SC       AR        524351.0      3
M-SC       XR        337654.0      4
M-MC       XY24      300633.0      2
H-MC       XY24      298264.0      3
X-LC       X40       289191.0      3
H-SC       XR        220220.0      3
           X40N      208026.0      4
H-LC       X40N      207214.0      4
M-SC       AR        187644.0      2
           OX40N     167650.0      5
X-MC       X40       160657.0      2
H-LC       X40       160590.0      3
M-LC       XY24      159049.0      2
L-SC       XR        150532.0      2
X-SC       XR        123341.0      1
X-LC       AR        121057.0      1
M-SC       XY25      118953.0      1
M-MC       XR        115255.0      1
X-MC       XY25      108149.0      1
M-LC       XR         99598.0      1
X-SC       SR         99409.0      1
X-LC       XY25       95158.0      3
H-SC       OX40N      85138.0      3
H-LC       AR         84469.0      1
M-SC       SR         83199.0      1
H-MC       XY25       82189.0      1
X-MC       XY24       80826.0      1
H-LC       X5K        74783.0      2
           X200       72628.0      1
M-SC       X40        71151.0      1
H-MC       AR         63173.0      1
L-SC       AR         58589.0      1
X-LC       X40N       54103.0      1
X-MC       X40N       54025.0      1
L-MC       XR         53786.0      1
H-SC       MH         53101.0      1
L-LC       XY25       48353.0      1
H-LC       XY24       42003.0      1
X-SC       OX40N      41036.0      2
           XY24       34069.0      1
H-MC       MH         30442.0      1
           X40        29285.0      1
           X40N       24608.0      1
L-SC       XY24       18935.0      1
X-MC       OX40N      18129.0      1
M-LC       XY25       14582.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
